In [1]:
import add_packages
import yaml, requests, re
from crewai import Agent, Task, Crew, Process
from pprint import pprint
from toolkit.crewai import tools as my_tools
from toolkit.langchain import chat_models

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
	configs_crewai = yaml.safe_load(f)

configs = configs_crewai

params = {
	"origin": "Ho Chi Minh City, Viet Nam",
	"travel_dates": "November 2024",
	"destination": "Da Lat, Viet Nam",
	"interests": "Hiking, Yoga, Sightseeing",
}

# Base

In [2]:
import re

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
  configs_crewai = yaml.safe_load(f)

configs = configs_crewai
configs_agents_traveling = configs_crewai["agents"]["traveling"]
configs_tasks_traveling = configs_crewai["tasks"]["traveling"]

params = {
	"origin": "Atlanta, GA",
	"travel_dates": "November 2024",
	"destination": "Thialand",
	"interests": "Hiking, Yoga, Sightseeing",
}

def _substitute_placeholders_in_configs(configs, params):
	pattern = r'%(\w+)%'

	def repl(match):
		placeholder = match.group(1)
		if placeholder.lower() in params:
			return str(params[placeholder.lower()])
		return match.group(0)

	return {
		key: {k: re.sub(pattern, repl, v) for k, v in value.items()}
		for key, value in configs.items()
	}


# Substitute placeholders in agent and task configurations
configs_agents_traveling_substituted = _substitute_placeholders_in_configs(
	configs_agents_traveling, params)
configs_tasks_traveling_substituted = _substitute_placeholders_in_configs(
	configs_tasks_traveling, params)

# Create agents
manager = Agent(
	**configs_agents_traveling_substituted["manager"],
	verbose=True,
	allow_delegation=True,
	llm=chat_models.chat_openai,
)

travel_agent = Agent(
	**configs_agents_traveling_substituted["travel_consultant"],
	tools=[
		my_tools.ToolsCalculator().calculate
	],
	max_iter=7,
	llm=chat_models.chat_openai,
)

city_selection_expert = Agent(
	**configs_agents_traveling_substituted["city_selection_expert"],
	verbose=True,
	tools=[
		my_tools.ToolsBrowser().search_duckduckgo,
		my_tools.ToolsContent().read_content
	],
	llm=chat_models.chat_openai,
)

local_tour_guide = Agent(
	**configs_agents_traveling_substituted["local_tour_guide"],
	verbose=True,
	tools=[
		my_tools.ToolsBrowser().search_duckduckgo,
		my_tools.ToolsContent().read_content
	],
	llm=chat_models.chat_openai,
)

quality_control_expert = Agent(
	**configs_agents_traveling_substituted["quality_control_expert"],
	verbose=True,
	tools=[] + my_tools.tools_human,
	llm=chat_models.chat_openai,
)

# Create tasks
manager_task = Task(
	**configs_tasks_traveling_substituted["manage"],
	agent=manager,
	async_execution=False,
)

identify_city = Task(
	**configs_tasks_traveling_substituted["identify_city"],
	agent=city_selection_expert,
	async_execution=False
)

gather_city_info = Task(
	**configs_tasks_traveling_substituted["gather_city_info"],
	agent=city_selection_expert,
	async_execution=False
)

plan_itinerary = Task(
	**configs_tasks_traveling_substituted["plan_itinerary"],
	agent=travel_agent,
	async_execution=False,
)

quality_control = Task(
	**configs_tasks_traveling_substituted["quality_control"],
	agent=quality_control_expert,
	context=[plan_itinerary],
	async_execution=False
)

# Forming the crew with a hierarchical process including the manager
crew = Crew(
	agents=[
		manager,
		travel_agent,
		city_selection_expert,
		local_tour_guide,
		quality_control_expert],
	tasks=[manager_task,
		   plan_itinerary,
		   identify_city,
		   gather_city_info,
		   quality_control],
	process=Process.hierarchical,
	manager_llm=chat_models.chat_openai,
	verbose=2,
)

TypeError: expected string or bytes-like object, got 'int'

In [ ]:

# Kick off the crew's work
results = crew.kickoff()

# Print the results
print("Crew Work Results:")
print(results)

# final

In [ ]:
class CustomCrew:
	def __init__(self, params, configs, task_name, tool_dict) -> None:
			self.params = params
			self.configs_agents: dict = configs["agents"][task_name]
			self.configs_tasks: dict = configs["tasks"][task_name]
			self.agents = {}
			self.tools = tool_dict
			self._create_agents()
			self.tasks = self._create_tasks()
			self._add_contexts()
			self.crew = self._create_crew()


	def _substitute_placeholders_in_configs(self, configs):
			if not configs:
					return {}
			
			pattern = r'%(\w+)%'
			def repl(match):
					placeholder = match.group(1)
					if placeholder.lower() in self.params:
							return str(self.params[placeholder.lower()])
					return match.group(0)
			
			return {
					key: {k: re.sub(pattern, repl, v) for k, v in value.items()}
					for key, value in configs.items()
			}

	def _create_agents(self):
		configs_agents_substituted = self._substitute_placeholders_in_configs(
				self.configs_agents)

		for agent_key, agent_config in configs_agents_substituted.items():
				tools_str = agent_config.get("tools")
				if tools_str == "x":
						agent_tools = []
				elif tools_str:
						agent_tools = [
								self.tools[tool_name.strip()] for tool_name in tools_str.split(",")
						]
				else:
						agent_tools = []

				agent_config["allow_delegation"] = agent_config.get(
						"allow_delegation", "false").lower() == "true"
				agent_config["tools"] = agent_tools

				self.agents[agent_key] = Agent(
						**agent_config,
						verbose=True,
						llm=chat_models.chat_openai,
				)
    
	def _create_tasks(self):
			configs_tasks_substituted = self._substitute_placeholders_in_configs(
							self.configs_tasks)
			tasks = {}

			for task_key, task_config in configs_tasks_substituted.items():
					task_config["async_execution"] = task_config.get(
									"async_execution", "false").lower() == "true"

					# Initialize agents for this task
					agent_name = task_config.get("agent", "")
					task_agent = self.agents[agent_name]
					
					task_config["agent"] = task_agent
					task_config.pop("context") # remove temporarily
     
					tasks[task_key] = Task(
						**task_config,
					)
			return tasks

	def _add_contexts(self):
		for task_key, task_config in self.configs_tasks.items():
				context_str = task_config.get("context", "")
				context_tasks = []
				if context_str == "x":
						context_tasks = []
				elif context_str:
						context_tasks = [self.tasks[task_name.strip()] for task_name in context_str.split(",")]

				self.tasks[task_key].context = context_tasks
 
	def _create_crew(self):
		return Crew(
			agents=list(self.agents.values()),
			tasks=list(self.tasks.values()),
			process=Process.hierarchical,
			manager_llm=chat_models.chat_openai,
			verbose=2
		)

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
	configs_crewai = yaml.safe_load(f)

configs = configs_crewai

params = {
	"origin": "Ho Chi Minh City, Viet Nam",
	"travel_dates": "November 2024",
	"destination": "Da Lat, Viet Nam",
	"interests": "Hiking, Yoga, Sightseeing",
}

task_name = "traveling"

tools_crew = {
	"calculator": my_tools.ToolsCalculator().calculate,
	"human": my_tools.tools_human,
	"scraper": my_tools.ToolsBrowser().scrape_and_summarize_website,
}

custom_crew = CustomCrew(params, configs, task_name, tools_crew)


In [ ]:
result = custom_crew.crew.kickoff()

In [ ]:
result

# test

In [2]:
class CustomCrew:
	def __init__(self, params, configs, task_name, tool_dict, llm) -> None:
			self.params = params
			self.configs_agents: dict = configs["agents"][task_name]
			self.configs_tasks: dict = configs["tasks"][task_name]
			self.agents = {}
			self.tools = tool_dict
			self.llm = llm
			self._create_agents()
			self.tasks = self._create_tasks()
			self._add_contexts()
			self.crew = self._create_crew()


	def _substitute_placeholders_in_configs(self, configs):
		if not configs:
				return {}

		pattern = r'%(\w+)%'
		def repl(match):
				placeholder = match.group(1)
				if placeholder.lower() in self.params:
						return str(self.params[placeholder.lower()])
				return match.group(0)

		return {
				key: {
						k: re.sub(pattern, repl, v) if isinstance(v, str) else v 
						for k, v in value.items()
				}
				for key, value in configs.items()
		}


	def _create_agents(self):
			configs_agents_substituted = self._substitute_placeholders_in_configs(
					self.configs_agents)

			for agent_key, agent_config in configs_agents_substituted.items():
					tools_str = agent_config.get("tools")
					if tools_str == "x":
							agent_tools = []
					elif tools_str:
							agent_tools = [
									self.tools[tool_name.strip()] for tool_name in tools_str.split(",")
							]
					else:
							agent_tools = []

					agent_config["allow_delegation"] = agent_config.get(
							"allow_delegation", "false").lower() == "true"
					agent_config["tools"] = agent_tools

					self.agents[agent_key] = Agent(
							**agent_config,
							verbose=True,
							llm=self.llm,
					)
    
	def _create_tasks(self):
			configs_tasks_substituted = self._substitute_placeholders_in_configs(
							self.configs_tasks)
			tasks = {}

			for task_key, task_config in configs_tasks_substituted.items():
					task_config["async_execution"] = task_config.get(
									"async_execution", "false").lower() == "true"

					# Initialize agents for this task
					agent_name = task_config.get("agent", "")
					task_agent = self.agents[agent_name]
					
					task_config["agent"] = task_agent
					task_config.pop("context") # remove temporarily
     
					tasks[task_key] = Task(
						**task_config,
					)
			return tasks

	def _add_contexts(self):
		for task_key, task_config in self.configs_tasks.items():
				context_str = task_config.get("context", "")
				context_tasks = []
				if context_str == "x":
						context_tasks = []
				elif context_str:
						context_tasks = [self.tasks[task_name.strip()] for task_name in context_str.split(",")]

				self.tasks[task_key].context = context_tasks
 
	def _create_crew(self):
		return Crew(
			agents=list(self.agents.values()),
			tasks=list(self.tasks.values()),
			process=Process.sequential,
			manager_llm=self.llm,
			verbose=2
		)

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
	configs_crewai = yaml.safe_load(f)

configs = configs_crewai

params = {
  "budget": "1,000,000 Viet Nam dong",
	"origin": "Ho Chi Minh City, Viet Nam",
	"travel_dates": "November 2024",
	"days": "5",
	"destination": "Da Lat, Viet Nam",
	"interests": "Hiking, Yoga, Sightseeing",
}

task_name = "traveling"

tools_crew = {
	"calculator": my_tools.ToolsCalculator().calculate,
	"human": my_tools.tools_human,
	"scraper": my_tools.ToolsBrowser().scrape_and_summarize_website,
	"searcher": my_tools.ToolsBrowser().search_serper,
}

custom_crew = CustomCrew(
	params=params, 
	configs=configs, 
	task_name=task_name, 
	tool_dict=tools_crew,
	llm=chat_models.chat_openai, # chat_openai, chat_groq_mixtral
)


In [3]:
result = custom_crew.crew.kickoff()

 [DEBUG]: == Working Agent: Travel Manager
 [INFO]: == Starting Task: Oversee the integration of research findings, trip suggestions, and quality control feedback to produce a 5-day travel itinerary specifically tailored to the user's information: 

- Budget: 1,000,000 Viet Nam dong
- Traveling from: Ho Chi Minh City, Viet Nam
- Travel destination: Da Lat, Viet Nam
- Traveler Interests: Hiking, Yoga, Sightseeing
- Trip Date: November 2024
- Travel Duration: 5
The final output should be a comprehensive with detailed per-day plans,  including budget, packing suggestions.



> Entering new CrewAgentExecutor chain...
I need to start by calculating the daily budget for the trip to Da Lat, Viet Nam.

Action: Make a calculation
Action Input: {"operation": "1000000 / 5"} 

200000.0

Final Answer: 

**Day 1: Arrival in Da Lat**
- Morning: Check into hotel, rest
- Afternoon: Visit Xuan Huong Lake, enjoy a leisurely walk
- Evening: Dinner at local restaurant

**Day 2: Hiking Adventure**
- Morning

In [4]:
result

'**Day 1: Arrival in Da Lat**\n- Arrival at Lien Khuong Airport, transfer to Dalat Palace Heritage Hotel.\n- Lunch at a local pho restaurant.\n- Visit the Linh Phuoc Pagoda.\n- Dinner at Ganesh Indian Restaurant.\n- Drinks at Maze Bar.\n\n**Day 2: Da Lat**\n- Breakfast at the hotel.\n- Visit the Crazy House.\n- Lunch at Hang Nga Guesthouse.\n- Explore the Dalat Flower Gardens.\n- Dinner at Art Cafe.\n- Nightlife at 100 Roofs Cafe.\n\n**Day 3: Da Lat**\n- Breakfast at the hotel.\n- Visit the Valley of Love.\n- Lunch at Thanh Thao Restaurant.\n- Explore the Dalat Market.\n- Dinner at Le Chalet Dalat.\n- Nightlife at Maze Bar.\n\n**Day 4: Da Lat**\n- Breakfast at the hotel.\n- Visit the Truc Lam Zen Monastery.\n- Lunch at Dalat Train Villa & Cafe.\n- Explore the Tuyen Lam Lake.\n- Dinner at Me Linh Coffee Garden.\n- Nightlife at 100 Roofs Cafe.\n\n**Day 5: Departure from Da Lat**\n- Breakfast at the hotel.\n- Visit the Dalat Cathedral.\n- Lunch at Ganesh Indian Restaurant.\n- Transfer to 

# Test

In [ ]:
result = eval('55')

number = 9
square_number = eval('number * number')
cmd = 'print(square_number)'
eval(cmd)

a = 1
cmd = 'a + 1'
eval(cmd)

# Info